In [37]:
import mne
import numpy as np
import os
from pathlib import Path
from EEG_quality_assessment.signal_metrics import SignalMetrics
import asrpy
import pyprep
import shutil

from eeg_fmri_cleaning.utils import read_raw_eeg
from eeg_fmri_cleaning.main import (
    clean_gradient,
    clean_bcg
)
import bids
from bids.layout import BIDSLayout

In [35]:
data_path = '/projects/EEG_FMRI/bids_eeg/BIDS/NEW/RAW'
layout = BIDSLayout(data_path)

In [38]:
l = list(layout.get_files())

In [39]:
entities

{'subject': '15',
 'session': '01',
 'task': 'tp',
 'run': 01,
 'suffix': 'eeg',
 'datatype': 'eeg',
 'extension': '.set'}

In [30]:
l[3]

'/projects/EEG_FMRI/bids_eeg/BIDS/NEW/RAW/sub-15/ses-01/eeg/sub-15_ses-01_task-tp_run-01_eeg.set'

In [33]:
layout.get_entities

{}

In [24]:
filename_base, filename_extension = os.path.splitext(l[3])

In [25]:
os.path.isfile(filename_base + '.json')

True

In [5]:
mne.channels.get_builtin_montages()

['standard_1005',
 'standard_1020',
 'standard_alphabetic',
 'standard_postfixed',
 'standard_prefixed',
 'standard_primed',
 'biosemi16',
 'biosemi32',
 'biosemi64',
 'biosemi128',
 'biosemi160',
 'biosemi256',
 'easycap-M1',
 'easycap-M10',
 'easycap-M43',
 'EGI_256',
 'GSN-HydroCel-32',
 'GSN-HydroCel-64_1.0',
 'GSN-HydroCel-65_1.0',
 'GSN-HydroCel-128',
 'GSN-HydroCel-129',
 'GSN-HydroCel-256',
 'GSN-HydroCel-257',
 'mgh60',
 'mgh70',
 'artinis-octamon',
 'artinis-brite23',
 'brainproducts-RNP-BA-128']

In [ ]:
file_list = layout.get(subject='01', 
           extension='set')
derivative_path = Path('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/DERIVATIVES')

def is_task_checker(BIDSFile):
    return 'checker' in BIDSFile.get_entities()['task']

def report(message, filename):
    with open(filename, 'a') as f:
        f.write(message)
        f.write('\n')

def cbin_cleaner(file):
    raw = read_raw_eeg(file.path)
    if file.get_entities()['task'] == 'checker':
        raw = clean_gradient(raw)
    raw = clean_bcg(raw)
    return raw

def asr_cleaner(raw):
    asr = asrpy.ASR()
    asr.fit(raw)
    raw = asr.transform(raw)
    return raw

def copy_sidecar(file, where_to_copy):
    filename_base, filename_extension = os.path.splitext(file.path)
    sidecar_filename = filename_base + '.json'
    if os.path.isfile(sidecar_filename):
        shutil.copyfile(sidecar_filename, where_to_copy)
     
def make_directory(file, derivative_path):
    entities = bids.layout.parse_file_entities(file)
    file_path = Path(derivative_path)
    file_path.joinpath(entities['subject'],
                       entities['session'],
                       entities['datatype'])
    file_path.mkdir(parents=True, exist_ok=True)
    
def pyprep_cleaner(raw):
    montage = mne.channels.make_standard_montage('easycap-M10')
    raw.apply_montage(montage)
    prep_params = {
        "ref_chs": "eeg",
        "reref_chs": "eeg",
        "line_freqs": np.arange(60, raw.info["sfreq"]/ 2, 60),
    }
    prep = pyprep.PrepPipeline(raw,
                               prep_params, 
                               channel_wise=True)
    prep.fit()
    raw = prep.transform(raw)
    return raw
        

In [ ]:
for file in file_list:
    if is_task_checker(file):
        try:
            raw = cbin_cleaner(file)
            
            
            pyprep_cleaner(raw)
            asr_cleaner(raw)
            report(file.path,
                f"{derivative_path}/processed.txt")
        except Exception as e:
            report(f"{file.path}: {str(e)}", 
                f"{derivative_path}/not_processed.txt")
            continue

In [3]:
a = (1,2)
try:
    a[0] = 3
except Exception as e:
    print(e)

'tuple' object does not support item assignment
